In [1]:
#Read in Libraries necessary for cleaning

In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.tokenize import sent_tokenize, word_tokenize, regexp_tokenize
import nltk
import seaborn as sns
from spacy.lang.en import English
from langdetect import detect
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import re

In [60]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [61]:
Titles = pd.read_csv('Total_Title_Data.csv')

C:\Users\Pupnsuds90\AppData\Local\Temp\ipykernel_6980\3995532456.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  Titles = pd.read_csv('Total_Title_Data.csv')


In [62]:
Titles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426870 entries, 0 to 426869
Data columns (total 9 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   EAN                   426870 non-null  int64  
 1   Title                 426870 non-null  object 
 2   Alt Vend              426870 non-null  object 
 3   Ingram Category Code  425121 non-null  object 
 4   MJR BISAC             426870 non-null  object 
 5   BISAC                 426870 non-null  object 
 6   List                  426870 non-null  float64
 7   Annotation            426870 non-null  object 
 8   Category              426870 non-null  int64  
dtypes: float64(1), int64(2), object(6)
memory usage: 29.3+ MB


In [63]:
Titles['EAN'] = Titles['EAN'].astype('str')
Titles['Title'] = Titles['Title'].astype('str')
Titles['Alt Vend'] = Titles['Alt Vend'].astype('str')
Titles['Ingram Category Code'] = Titles['Ingram Category Code'].astype('str')
Titles['MJR BISAC'] = Titles['MJR BISAC'].astype('str')
Titles['Annotation'] = Titles['Annotation'].astype('str')
Titles['Category'] = Titles['Category'].astype('str')

In [64]:
Titles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426870 entries, 0 to 426869
Data columns (total 9 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   EAN                   426870 non-null  object 
 1   Title                 426870 non-null  object 
 2   Alt Vend              426870 non-null  object 
 3   Ingram Category Code  426870 non-null  object 
 4   MJR BISAC             426870 non-null  object 
 5   BISAC                 426870 non-null  object 
 6   List                  426870 non-null  float64
 7   Annotation            426870 non-null  object 
 8   Category              426870 non-null  object 
dtypes: float64(1), object(8)
memory usage: 29.3+ MB


In [65]:
Titles.head()

,EAN,Title,Alt Vend,Ingram Category Code,MJR BISAC,BISAC,List,Annotation,Category
0,9781803143057,Murder at the Country Club: An absolutely unpu...,6072814,FR,FIC,FIC027110,9.99,<p><strong>Kitty Underhay is playing doubles.....,1
1,9798440995017,From Sandpaper To Silk (Book One) ...,6107230,OP,OCC,OCC000000,15.00,"This is a very suspenseful, engaging, and rive...",1
2,9798885362184,Jason Lang & the iTunes Scandals: The Real-Lif...,6110949,BA,BIO,BIO000000,9.99,<p>Real Life Story About the Gilgo Beach Murde...,1
3,9781636160887,Izzie's Incredible Imagination ...,9269037,CL,JUV,JUV001000,9.99,<p>Whether you are floating in space or riding...,1
4,9781803810324,"Zeppelins in the Searchlights, Sisters of Merc...",6027623,BA,BIO,BIO026000,9.50,"<p>Born in 1909 in Hull, Bernard recalls how h...",1


In [66]:
from bs4 import BeautifulSoup

def remove_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

Titles['Annotation Text'] = Titles['Annotation'].apply(remove_html_tags)

C:\Users\Pupnsuds90\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [67]:
Titles.head()

,EAN,Title,Alt Vend,Ingram Category Code,MJR BISAC,BISAC,List,Annotation,Category,Annotation Text
0,9781803143057,Murder at the Country Club: An absolutely unpu...,6072814,FR,FIC,FIC027110,9.99,<p><strong>Kitty Underhay is playing doubles.....,1,Kitty Underhay is playing doubles... with deat...
1,9798440995017,From Sandpaper To Silk (Book One) ...,6107230,OP,OCC,OCC000000,15.00,"This is a very suspenseful, engaging, and rive...",1,"This is a very suspenseful, engaging, and rive..."
2,9798885362184,Jason Lang & the iTunes Scandals: The Real-Lif...,6110949,BA,BIO,BIO000000,9.99,<p>Real Life Story About the Gilgo Beach Murde...,1,Real Life Story About the Gilgo Beach Murders ...
3,9781636160887,Izzie's Incredible Imagination ...,9269037,CL,JUV,JUV001000,9.99,<p>Whether you are floating in space or riding...,1,Whether you are floating in space or riding a ...
4,9781803810324,"Zeppelins in the Searchlights, Sisters of Merc...",6027623,BA,BIO,BIO026000,9.50,"<p>Born in 1909 in Hull, Bernard recalls how h...",1,"Born in 1909 in Hull, Bernard recalls how he s..."


In [68]:
Titles['Annotation Text'] = Titles['Annotation Text'].astype('str')

In [69]:
# Define the regex pattern
pattern = r'&#\d+;'

# Create a new dataset for titles matching the pattern
new_dataset = Titles[Titles['Title'].str.contains(pattern, regex=True)].copy()

# Remove the matching titles from the original dataset
Titles = Titles[~Titles['Title'].str.contains(pattern, regex=True)].reset_index(drop=True)

# Print the new dataset containing titles matching the pattern
print("New Dataset:")
print(new_dataset)

# Print the updated original dataset without titles matching the pattern
print("Updated Original Dataset:")
print(Titles)

New Dataset:
                  EAN                                              Title  \
144     9781685364816  The Song of &#23433;&#29738; An-Chee          ...   
154     9789948817444  &#1571;&#1579;&#1610;&#1585;&#1610;           ...   
162     9781647841553  &#26368;&#20986;&#21517;&#30340;&#19968;&#3000...   
177     9789948817260  &#1587;&#1576;&#1578;&#1575;&#1580;&#1585;&#15...   
188     9789948817482  &#1575;&#1604;&#1576;&#1585;&#1607;&#1575;&#16...   
...               ...                                                ...   
421445  9791188195107  Best Korean Short Stories Collection &#45824;&...   
421447  9791188195282  Best Korean Short Stories Collection 2 &#45824...   
422204  9781958313084  Muslimische Frauen und der Hijab-Schleier: Unt...   
422312  9781956555257  Trace and Color: Learning Collection Ages 3-6 ...   
422628  9781772280333  Navzdy Se Mnou: O Mém U&#268;iteli Rabasovi   ...   

       Alt Vend Ingram Category Code MJR BISAC      BISAC   List  \
144   

In [70]:
new_dataset

,EAN,Title,Alt Vend,Ingram Category Code,MJR BISAC,BISAC,List,Annotation,Category,Annotation Text
144,9781685364816,The Song of &#23433;&#29738; An-Chee ...,9223758,MU,MUS,MUS000000,8.99,<p>Anchee Escapes from the Chinese dynasty pas...,1,Anchee Escapes from the Chinese dynasty past t...
154,9789948817444,&#1571;&#1579;&#1610;&#1585;&#1610; ...,6112990,HE,SEL,SEL027000,7.95,&#1604;&#1604;&#1602;&#1605;&#1585; &#1585;&#1...,1,للقمر رمزيةٌ خاصة لفهم أنَّ أرواحَنا لا تفنى ح...
162,9781647841553,&#26368;&#20986;&#21517;&#30340;&#19968;&#3000...,9170656,HU,HUM,HUM003000,19.99,<p>&#20940;&#40718;&#24180;&#65292;&#20013;&#2...,1,凌鼎年，中國作家協會會員、世界華文微型小說研究會會長、亞洲微電影學院客座教授，美國紐約商務出...
177,9789948817260,&#1587;&#1576;&#1578;&#1575;&#1580;&#1585;&#15...,6112990,FM,FIC,FIC022060,28.95,&#1605;&#1575; &#1586;&#1575;&#1604;&#1578; (&...,1,ما زالت (ليليث) تجوس بين الظِّلال، ما زال (لوس...
188,9789948817482,&#1575;&#1604;&#1576;&#1585;&#1607;&#1575;&#16...,6112990,CL,JNF,JNF049000,14.95,&#1585;&#1581;&#1604;&#1577; &#1605;&#1606;&#1...,1,رحلة منطقية مِن البداية، مِن أصول المنطق والتف...
...,...,...,...,...,...,...,...,...,...,...
421445,9791188195107,Best Korean Short Stories Collection &#45824;&...,9037057,LF,FOR,FOR015000,12.95,<p>Meet the very finest of Korean short storie...,3,Meet the very finest of Korean short stories. ...
421447,9791188195282,Best Korean Short Stories Collection 2 &#45824...,9037057,LF,FOR,FOR015000,12.95,<p>Meet the very finest of Korean short storie...,3,Meet the very finest of Korean short stories. ...
422204,9781958313084,Muslimische Frauen und der Hijab-Schleier: Unt...,9184014,RS,REL,REL037000,9.90,<p>In den Medien und in der Gesellschaft wird ...,3,In den Medien und in der Gesellschaft wird die...
422312,9781956555257,Trace and Color: Learning Collection Ages 3-6 ...,9242563,CL,JNF,JNF006020,9.29,<p><strong>Perfect tool for home schooling and...,3,Perfect tool for home schooling and having fun...


In [71]:
#Remove 'middle' category, representing 'uncertain' classifications
Titles_Binary = Titles[Titles['Category'].isin(['1','3'])]

In [72]:
Titles_Binary = Titles_Binary[['Title', 'Ingram Category Code', 'MJR BISAC', 'BISAC', 'Annotation Text']]

In [73]:
tokenizer = AutoTokenizer.from_pretrained("papluca/xlm-roberta-base-language-detection")
pipe = pipeline("text-classification", model="papluca/xlm-roberta-base-language-detection", tokenizer = tokenizer)

In [74]:
Titles_Test = Titles_Binary.loc[0:5,]

In [78]:
Titles_Binary.dropna(inplace=False)

,Title,Ingram Category Code,MJR BISAC,BISAC,Annotation Text
0,Murder at the Country Club: An absolutely unpu...,FR,FIC,FIC027110,Kitty Underhay is playing doubles... with deat...
1,From Sandpaper To Silk (Book One) ...,OP,OCC,OCC000000,"This is a very suspenseful, engaging, and rive..."
2,Jason Lang & the iTunes Scandals: The Real-Lif...,BA,BIO,BIO000000,Real Life Story About the Gilgo Beach Murders ...
3,Izzie's Incredible Imagination ...,CL,JUV,JUV001000,Whether you are floating in space or riding a ...
4,"Zeppelins in the Searchlights, Sisters of Merc...",BA,BIO,BIO026000,"Born in 1909 in Hull, Bernard recalls how he s..."
...,...,...,...,...,...
421883,Zoe and the I Can't Monkey ...,CL,JUV,JUV039140,When Zoe decides to take the training wheels o...
421884,Zoe and the I Can't Monkey ...,CL,JUV,JUV039140,When Zoe decides to take the training wheels o...
421885,Zombie Apocalypse Call Center ...,FC,FIC,FIC002000,"Who are you going to call, to survive the zomb..."
421886,Zone Diet: The Most Effective Weight Loss Tool...,CW,CKB,CKB000000,"This book analyzes the blue zone diet, includi..."


In [80]:
from transformers import pipeline, AutoTokenizer
import pandas as pd
import torch
import torch.nn.functional as F
from tqdm import tqdm

In [ ]:
Titles_Binary['Language'] = ""
Titles_Binary['Score'] = ""

# Batch processing parameters
batch_size = 1500  # Adjust the batch size based on your system's capacity

# Calculate the total number of batches
num_batches = (len(Titles_Binary) - 1) // batch_size + 1

# Iterate over the dataset in batches
for batch_index, start_index in tqdm(enumerate(range(0, len(Titles_Binary), batch_size), 1), total=num_batches, desc='Processing'):
    end_index = min(start_index + batch_size, len(Titles_Binary))
    batch_texts = Titles_Binary['Annotation Text'][start_index:end_index].tolist()

    # Encode the batch of texts
    encoded_inputs = tokenizer.batch_encode_plus(
        batch_texts,
        truncation=True,
        max_length=5,
        padding='max_length',
        return_tensors='pt'
    )

    # Make predictions for the batch
    outputs = pipe.model(**encoded_inputs)
    logits = outputs.logits
    predicted_class_indices = logits.argmax(dim=1).tolist()

    # Apply softmax to obtain confidence scores
    scores = F.softmax(logits, dim=1)

    # Update the corresponding rows in the dataset
    for i, index in enumerate(range(start_index, end_index)):
        predicted_class_idx = predicted_class_indices[i]
        language = pipe.model.config.id2label[predicted_class_idx]
        score = scores[i][predicted_class_idx].item()

        Titles_Binary.at[index, 'Language'] = language
        Titles_Binary.at[index, 'Score'] = score

# Print the updated dataset
print(Titles_Binary)

Processing:  42%|█████████████████████████▍                                   | 110/264 [21:32:02<30:35:04, 714.96s/it]

In [28]:
Titles_Binary['Language'] = ""
Titles_Binary['Score'] = ""

# Batch processing parameters
batch_size = 100  # Adjust the batch size based on your system's capacity
chunk_size = 10000  # Adjust the chunk size based on the available memory

# Calculate the total number of chunks
num_chunks = (len(Titles_Binary) - 1) // chunk_size + 1

# Iterate over the dataset in chunks
for chunk_index, start_index in tqdm(enumerate(range(0, len(Titles_Binary), chunk_size), 1), total=num_chunks, desc='Processing Chunks'):
    end_index = min(start_index + chunk_size, len(Titles_Binary))
    chunk_texts = Titles_Binary['Annotation Text'][start_index:end_index].tolist()

    # Calculate the total number of batches within the current chunk
    num_batches = (len(chunk_texts) - 1) // batch_size + 1

    # Iterate over the chunk in batches
    for batch_index, batch_start_index in enumerate(range(0, len(chunk_texts), batch_size), 1):
        batch_end_index = min(batch_start_index + batch_size, len(chunk_texts))
        batch_texts = chunk_texts[batch_start_index:batch_end_index]

        # Encode the batch of texts
        encoded_inputs = tokenizer.batch_encode_plus(
            batch_texts,
            truncation=True,
            max_length=15,
            padding='max_length',
            return_tensors='pt'
        )

        # Make predictions for the batch
        outputs = pipe.model(**encoded_inputs)
        logits = outputs.logits
        predicted_class_indices = logits.argmax(dim=1).tolist()

        # Apply softmax to obtain confidence scores
        scores = F.softmax(logits, dim=1)

        # Update the corresponding rows in the dataset
        for i, index in enumerate(range(start_index + batch_start_index, start_index + batch_end_index)):
            predicted_class_idx = predicted_class_indices[i]
            language = pipe.model.config.id2label[predicted_class_idx]
            score = scores[i][predicted_class_idx].item()

            Titles_Binary.at[index, 'Language'] = language
            Titles_Binary.at[index, 'Score'] = score

# Print the updated dataset
print(Titles_Binary)

Processing Chunks:   0%|                                                                        | 0/40 [01:27<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
#Empty Columns for Saving Language Prediction and Score
Titles_Test['Language'] = ""
Titles_Test['Score'] = ""

for index, row in tqdm(Titles.iterrows(), total=len(Titles_Test), desc='Processing'):
    text = row['Annotation Text']

    # encode_plus to add additional qualifiers to speed up latency
    encoded_input = tokenizer.encode_plus(
        text,
        truncation=True,
        max_length=5,
        padding='max_length',
        return_tensors='pt'
    )

    # Make predictions using the encoded input
    output = pipe.model(**encoded_input)
    logits = output.logits
    predicted_class_idx = logits.argmax(dim=1).item()

    # Apply softmax to obtain confidence scores
    scores = F.softmax(logits, dim=1)[0]

    # Get the language label and confidence score from the pipeline's label list and scores
    language = pipe.model.config.id2label[predicted_class_idx]
    score = scores[predicted_class_idx].item()

    # Update the corresponding row in the dataset
    Titles_Test.at[index, 'language'] = language
    Titles_Test.at[index, 'score'] = score


Titles_Test

In [ ]:
predicted_language

In [ ]:
Titles.loc[470,'Annotation Text']

In [ ]:
Titles['PredictedLanguage'].value_counts()

In [ ]:
import re

#Identify Titles in the datset with the code pattern and remove them from the dataframe
pattern = r'&#\d+;'
Titles = Titles[~Titles['Title'].str.contains(pattern, regex=True)]

#Reset the index of the dataframe
Titles = Titles.reset_index(drop=True)